In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import joblib
import datetime

In [2]:
train = pd.read_csv("cleaned_train.csv")
test = pd.read_csv("cleaned_test.csv")

/tmp/ipykernel_387/1718243997.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv("cleaned_train.csv")
/tmp/ipykernel_387/1718243997.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv("cleaned_test.csv")


In [3]:
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,...,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Year,Month,WeekOfYear,DayOfMonth,DistanceBucket
0,1,5,2015-07-31,5263,555,1,1,0,1,c,...,2008.0,0,0.0,0.0,NaN,2015,7,31,31,500-1500
1,2,5,2015-07-31,6064,625,1,1,0,1,a,...,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",2015,7,31,31,500-1500
2,3,5,2015-07-31,8314,821,1,1,0,1,a,...,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",2015,7,31,31,10000+
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,...,2009.0,0,0.0,0.0,NaN,2015,7,31,31,500-1500
4,5,5,2015-07-31,4822,559,1,1,0,1,a,...,2015.0,0,0.0,0.0,NaN,2015,7,31,31,10000+


In [4]:
test.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,...,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Year,Month,WeekOfYear,DayOfMonth
0,1,1,4,2015-09-17,1.0,1,0,0,c,a,...,9.0,2008.0,0,0.0,0.0,NaN,2015,9,38,17
1,2,3,4,2015-09-17,1.0,1,0,0,a,a,...,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",2015,9,38,17
2,3,7,4,2015-09-17,1.0,1,0,0,a,c,...,4.0,2013.0,0,0.0,0.0,NaN,2015,9,38,17
3,4,8,4,2015-09-17,1.0,1,0,0,a,a,...,10.0,2014.0,0,0.0,0.0,NaN,2015,9,38,17
4,5,9,4,2015-09-17,1.0,1,0,0,a,c,...,8.0,2000.0,0,0.0,0.0,NaN,2015,9,38,17


In [5]:
train.isnull().sum()

Store                             0
DayOfWeek                         0
Date                              0
Sales                             0
Customers                         0
Open                              0
Promo                             0
StateHoliday                      0
SchoolHoliday                     0
StoreType                         0
Assortment                        0
CompetitionDistance               0
CompetitionOpenSinceMonth         0
CompetitionOpenSinceYear          0
Promo2                            0
Promo2SinceWeek                   0
Promo2SinceYear                   0
PromoInterval                508031
Year                              0
Month                             0
WeekOfYear                        0
DayOfMonth                        0
DistanceBucket                    0
dtype: int64

In [6]:
test.isnull().sum()

Id                               0
Store                            0
DayOfWeek                        0
Date                             0
Open                             0
Promo                            0
StateHoliday                     0
SchoolHoliday                    0
StoreType                        0
Assortment                       0
CompetitionDistance              0
CompetitionOpenSinceMonth        0
CompetitionOpenSinceYear         0
Promo2                           0
Promo2SinceWeek                  0
Promo2SinceYear                  0
PromoInterval                17232
Year                             0
Month                            0
WeekOfYear                       0
DayOfMonth                       0
dtype: int64

In [7]:
train['PromoInterval'].fillna("None", inplace=True)
test['PromoInterval'].fillna("None", inplace=True)

/tmp/ipykernel_387/3783106480.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train['PromoInterval'].fillna("None", inplace=True)
/tmp/ipykernel_387/3783106480.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try

In [9]:
train.isnull().sum()

Store                        0
DayOfWeek                    0
Date                         0
Sales                        0
Customers                    0
Open                         0
Promo                        0
StateHoliday                 0
SchoolHoliday                0
StoreType                    0
Assortment                   0
CompetitionDistance          0
CompetitionOpenSinceMonth    0
CompetitionOpenSinceYear     0
Promo2                       0
Promo2SinceWeek              0
Promo2SinceYear              0
PromoInterval                0
Year                         0
Month                        0
WeekOfYear                   0
DayOfMonth                   0
DistanceBucket               0
dtype: int64

In [10]:
# datetime features
train['Date'] = pd.to_datetime(train['Date'])
train['Weekday'] = train['Date'].dt.dayofweek  # 0=Monday, 6=Sunday
train['IsWeekend'] = (train['Weekday'] >= 5).astype(int)
train['Month'] = train['Date'].dt.month
train['Quarter'] = train['Date'].dt.quarter
train['Year'] = train['Date'].dt.year
train['IsBeginningOfMonth'] = (train['Date'].dt.day <= 10).astype(int)
train['IsMidMonth'] = ((train['Date'].dt.day > 10) & (train['Date'].dt.day <= 20)).astype(int)
train['IsEndOfMonth'] = (train['Date'].dt.day > 20).astype(int)

test['Date'] = pd.to_datetime(test['Date'])
test['Weekday'] = test['Date'].dt.dayofweek  # 0=Monday, 6=Sunday
test['IsWeekend'] = (test['Weekday'] >= 5).astype(int)
test['Month'] = test['Date'].dt.month
test['Quarter'] = test['Date'].dt.quarter
test['Year'] = test['Date'].dt.year
test['IsBeginningOfMonth'] = (test['Date'].dt.day <= 10).astype(int)
test['IsMidMonth'] = ((test['Date'].dt.day > 10) & (test['Date'].dt.day <= 20)).astype(int)
test['IsEndOfMonth'] = (test['Date'].dt.day > 20).astype(int)


In [11]:
# holiday features
train['DaysToHoliday'] = train.groupby('Store')['StateHoliday'].transform(lambda x: (x.shift(-1) != '0').cumsum())
train['DaysAfterHoliday'] = train.groupby('Store')['StateHoliday'].transform(lambda x: (x != '0').cumsum())

In [12]:
test['DaysToHoliday'] = test.groupby('Store')['StateHoliday'].transform(lambda x: (x.shift(-1) != '0').cumsum())
test['DaysAfterHoliday'] = test.groupby('Store')['StateHoliday'].transform(lambda x: (x != '0').cumsum())

In [13]:
# promo features
train['Promo2Active'] = ((train['Promo2'] == 1) & (train['Month'].isin([2, 5, 8, 11]))).astype(int)
test['Promo2Active'] = ((test['Promo2'] == 1) & (test['Month'].isin([2, 5, 8, 11]))).astype(int)

In [15]:
# lag features
train['SalesLag7'] = train.groupby('Store')['Sales'].shift(7)
train['SalesLag30'] = train.groupby('Store')['Sales'].shift(30)
train['SalesChange'] = train['Sales'] - train['SalesLag7']

In [16]:
# One-hot encode categorical variables
categorical_features = ['StateHoliday', 'StoreType', 'Assortment']
train = pd.get_dummies(train, columns=categorical_features, drop_first=True)

In [21]:
# Select numeric columns to scale
numeric_features = ['CompetitionDistance', 'SalesLag7', 'SalesLag30', 'DaysToHoliday', 'DaysAfterHoliday']

# Apply StandardScaler
scaler = StandardScaler()
train[numeric_features] = scaler.fit_transform(train[numeric_features])

In [26]:
train.to_csv("preprocessed_train.csv", index=False)
test.to_csv("preprocessed_test.csv", index=False)